In [2]:
%load_ext google.cloud.bigquery

# Working with Arrays

## Arrays can preserve order

In the following query, ORDER BY works but if we need to save the data in an intermediary table for 
downstream analysis there is not guarantee of preserving ordering when reading it back from the table

In [3]:
%%bigquery

SELECT
  bike_id,
  COUNT(*) AS num_trips
FROM
  dataflow-templates-327714.bigquery_examples.cycle_hire
GROUP BY
  bike_id ORDER BY num_trips DESC
LIMIT
  100

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 68.49rows/s]


,bike_id,num_trips
0,12925,2922
1,12841,2871
2,13071,2860
3,12926,2854
4,12991,2829
...,...,...
95,10874,2600
96,12430,2599
97,11759,2598
98,12839,2598


On relatively small results related to reuse in the same query, Arrays can be a better option if ordering is
important for the subsequent analysis steps. **One solution would be to store the result into a single row by aggregating the result into an array and moving the order by clause to ARRAY_AGG**.

In [4]:
%%bigquery
WITH
  numtrips AS (
  SELECT
    bike_id AS id,
    COUNT(*) AS num_trips
  FROM
    dataflow-templates-327714.bigquery_examples.cycle_hire
  GROUP BY
    bike_id )
SELECT
  ARRAY_AGG(STRUCT(id,
      num_trips)
  ORDER BY
    num_trips DESC
  LIMIT
    100) AS bike
FROM
  numtrips

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/rows]


,bike
0,"[{'id': 12925, 'num_trips': 2922}, {'id': 1284..."


You can think of an array of struct as mini table stored in a single row.

## Storing repeated fields

Example use case of a repeated field, is 1 to n data to be stored on the same row without external tables to JOIN 
e.g. 1 organisation having multiple tax filing records

In [9]:
%%bigquery
SELECT
  ein,
  tax_pd, 
  subseccd
FROM
  `bigquery-public-data.irs_990.irs_990_2015`
WHERE
  ein BETWEEN '200'
  AND '399'
LIMIT
  8;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.59rows/s]


,ein,tax_pd,subseccd
0,390123480,201412,8
1,390123480,201312,8
2,382227794,201406,6
3,361493430,201412,8
4,361493430,201312,8
5,366066772,201412,14
6,376028123,201412,14
7,362154936,201412,9


In [8]:
%%bigquery
SELECT
  ein,
  ARRAY_AGG(STRUCT(elf,
      tax_pd,
      subseccd)) AS filing
FROM
  `bigquery-public-data.irs_990.irs_990_2015`
WHERE
  ein BETWEEN '200'
  AND '399'
GROUP BY
  ein
LIMIT
  8;

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.51rows/s]


,ein,filing
0,390123480,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 8}..."
1,382227794,"[{'elf': 'E', 'tax_pd': 201406, 'subseccd': 6}]"
2,361493430,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 8}..."
3,366066772,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 14}]"
4,376028123,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 14}]"
5,362154936,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 9}]"
6,230961140,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 14}]"
7,381350130,"[{'elf': 'E', 'tax_pd': 201412, 'subseccd': 14}]"


## Generating data

Example of a list of summer days

In [11]:
%%bigquery

SELECT
  GENERATE_DATE_ARRAY('2019-06-23', '2019-08-22', INTERVAL 10 DAY) AS summer

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/rows]


,summer
0,"[2019-06-23, 2019-07-03, 2019-07-13, 2019-07-2..."


This can be converted back to table via **UNNEST**

In [12]:
%%bigquery

WITH
  days AS (
  SELECT
    GENERATE_DATE_ARRAY('2019-06-23', '2019-08-22', INTERVAL 10 DAY) AS summer)
SELECT
  summer_day
FROM
  days,
  UNNEST(summer) AS summer_day

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.30rows/s]


,summer_day
0,2019-06-23
1,2019-07-03
2,2019-07-13
3,2019-07-23
4,2019-08-02
5,2019-08-12
6,2019-08-22


### Array data can be hard coded when needed as well

In [13]:
%%bigquery

SELECT ['Michael', 'Jordan', 'Graham'] as people

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.41s/rows]


,people
0,"[Michael, Jordan, Graham]"


### Combining data from two arrays into a table

In [15]:
%%bigquery

WITH
  days AS (
  SELECT
    GENERATE_DATE_ARRAY('2019-06-23', '2019-08-22', INTERVAL 10 DAY) AS summer,
    ['Michael',
    'Jordan',
    'Graham'] AS people )
SELECT
  summer[ORDINAL(dayno)] AS summer_day,
  people[
OFFSET
  (MOD(dayno, ARRAY_LENGTH(people)))] AS people
FROM
  days,
  UNNEST(GENERATE_ARRAY(1, ARRAY_LENGTH(summer), 1)) dayno
ORDER BY
  summer_day ASC

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  5.28rows/s]


,summer_day,people
0,2019-06-23,Jordan
1,2019-07-03,Graham
2,2019-07-13,Michael
3,2019-07-23,Jordan
4,2019-08-02,Graham
5,2019-08-12,Michael
6,2019-08-22,Jordan


** Both OFFSET and ORDINAL functions have similar use case, difference is OFFSET starts with 0, ORDINAL with 1**

## Concatenating Arrays

In [16]:
%%bigquery

SELECT ARRAY_CONCAT(
    GENERATE_DATE_ARRAY('2019-03-23', '2019-06-22', INTERVAL 20 DAY),
    GENERATE_DATE_ARRAY('2019-08-23', '2019-11-22', INTERVAL 20 DAY)
) AS all_season

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.44s/rows]


,all_season
0,"[2019-03-23, 2019-04-12, 2019-05-02, 2019-05-2..."


## Other useful functions

### ARRAY_TO_STRING (useful for debugging)

In [18]:
%%bigquery

-- * is used as a seperator
-- na is set as a placeholder for NULL value

SELECT ARRAY_TO_STRING(['A', 'B', NULL, 'D'], '*', 'na') AS arr

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.36s/rows]


,arr
0,A*B*na*D
